# Breast MRI QA

This notebook allows a user to obtain MRI studies from an Orthanc DICOM server and apply a range of QA tests. The notebook acts as an interface to the `breast_mri_qa` package. Please read the instructions preceding each code cell, make any neccesary changes, and then execute the code before moving on to the next code block. 

In [ ]:
from breast_mri_qa import fetch, organise, measure

Define the Orthanc server information and login credentials.

In [ ]:
host = '139.59.186.101' # <- EDITABLE
port = 80 # <- EDITABLE
user = 'orthanc' # <- EDITABLE
passwd = 'orthanc' # <- EDITABLE

Create a 'Fetcher' object which can query data at the patient, study and series level as well as return an instance. 

In [ ]:
fetcher = fetch.Fetcher(host=host, port=port, user=user, passwd=passwd)

## >>> Input required
Specify string `patient_name` used to search Orthanc server to find all studies associated with patients with a name matching that string.

In [ ]:
patient_name = 'BREAST*' # <- EDITABLE
n_most_recent = 5 # <- EDITABLE

In [ ]:
studies = fetcher.get_n_most_recent_study_details(patient_name=patient_name, n_most_recent=n_most_recent)
studyuids = [study['StudyUID'] for study in studies]
studies

## >>> Input required

Select the StudyUID you want y specifying the value of `selected_study` (remember they are zero-indexed). e.g. `studyuids[0]` for the first item, or `studyuids[1]` for the second item.

In [ ]:
selected_study = 3 # <- EDITABLE

In [ ]:
studyuid = studyuids[selected_study]
study = studies[selected_study]
seriesuids = fetcher.get_series(studyuid)
instances = list(filter(lambda x: x is not None, (fetcher.get_valid_image_instance(studyuid, uid) for uid in seriesuids)))

Check whether the list of instances contains a full protocol. `rules` contains the info required to match images to the protocol. Each protocol image rule must be a 3-tuple which has the following elements in this exact order:
- Name used to reference the image
- The name of the function used to determine whether the image is of the required type
- The string used to identify the image type (e.g. whether it's a SNR or FSE image)

In [ ]:
rules = [
    ('snr_acquisition_one', 'is_snr', 'TEST'),
    ('snr_acquisition_two', 'is_snr', 'TEST'),
    ('spir_water', 'is_spir_water_fse', 'SPIR WATER'),
    ('spir_fat', 'is_spir_fat_fse', 'SPIR FAT'),
    ('spair_water', 'is_spair_water_fse', 'SPAIR WATER'),
    ('spair_fat', 'is_spair_fat_fse', 'SPAIR FAT'),
    ('coil_one_acquisition_one', 'is_coil_one', 'COIL 1'),
    ('coil_two_acquisition_two', 'is_coil_one', 'COIL 1'),
    ('coil_one_acquisition_two', 'is_coil_two', 'COIL 2'),
    ('coil_two_acquisition_one', 'is_coil_two', 'COIL 2'),
    ('coil_three_acquisition_one', 'is_coil_three', 'COIL 3'),
    ('coil_three_acquisition_two', 'is_coil_three', 'COIL 3'),
    ('coil_four_acquisition_one', 'is_coil_four', 'COIL 4'),
    ('coil_four_acquisition_two', 'is_coil_four', 'COIL 4'),
    ('coil_five_acquisition_one', 'is_coil_five', 'COIL 5'),
    ('coil_five_acquisition_two', 'is_coil_five', 'COIL 5'),
    ('coil_six_acquisition_one', 'is_coil_six', 'COIL 6'),
    ('coil_six_acquisition_two', 'is_coil_six', 'COIL 6'),
    ('coil_seven_acquisition_one', 'is_coil_seven', 'COIL 7'),
    ('coil_seven_acquisition_two', 'is_coil_seven', 'COIL 7'),
]
protocol = organise.Protocol(rules)
missing_instances = protocol.assign_instances_to_protocol(instances)
assert not missing_instances, missing_instances

If you made it this far without an error then the study contains all the acquisitions required to do the QA - woohoo!

In [ ]:
get_mid_slice = measure.get_mid_slice
images = protocol.dict_protocol_instances
study['SNR'] = measure.snr(get_mid_slice(images['snr_acquisition_one']), get_mid_slice(images['snr_acquisition_two']))
study['SPIR-FSE'] = measure.fse(get_mid_slice(images['spir_fat']), get_mid_slice(images['spir_water']))
study['SPAIR-FSE'] = measure.fse(get_mid_slice(images['spair_fat']), get_mid_slice(images['spair_water']))
num_to_str = {'1':'one','2':'two','3':'three','4':'four','5':'five','6':'six','7':'seven'}
identifier = 'coil_{}_acquisition_one'
for row in enumerate(num_to_str.items()):
    study[identifier.format(row[1][1])] = '{:.2f}'.format(get_mid_slice(images[identifier.format(row[1][1])]).mean())

Output results

In [ ]:
study['StationName'] = instances[0]['StationName']
study['StudyDescription'] = instances[0]['StudyDescription']
study['StationName'] = instances[0]['StationName']
study['PatientID'] = instances[0]['PatientID']
study['MagneticFieldStrength'] = instances[0]['MagneticFieldStrength']
study